In [1]:
from obspy import UTCDateTime
import obspy
from obspy.io.sac import SACTrace
from obspy.taup import TauPyModel
from obspy import read_inventory
from obspy.clients.iris import Client
client = Client()
import os
import glob 
from IPython.display import display, clear_output
import re

In [2]:
def read_evt(evtf):
    """
    read event information
    """
    with open(evtf, 'r') as f:
        lines = f.readlines()[1:]
        events = []
        dnames = []
        for line in lines:
            line = line.strip()
            temp = line.split(',')
            t = UTCDateTime(temp[0])
            lat, lon = float(temp[1]), float(temp[2])
            depth = float(temp[3])
            dpu = temp[4]
            mag = float(temp[5])
            magt = temp[6]
            events.append([t, lat, lon, depth, dpu, mag, magt])
            dnames.append("".join(temp[0].split("T")[0].split("-")) + "".join("".join("".join(temp[0].split("T")[1].split("Z")).split(".")).split(":")))
        return dnames, events

In [3]:

def mseed2sac(data_root, data_out, data_staxml, dnames, events):
    """
    convert miniseed to SAC
    """
    total_run_num = len(events)
    run_num = 0
    for evt in zip(dnames, events):
        
        outd = os.path.join(data_out, evt[0])
        if os.path.exists(outd):
            continue
        else:
            os.makedirs(outd)

            fpath = os.path.join(data_root, evt[0], '*.mseed')
            try:
                #st is the mseed file
                st = obspy.read(fpath)
            except FileNotFoundError:
                print("Missing File: %s" % fpath)
                continue
            
            # get the staxml
            run_num += 1
            print(f"run {run_num} of {total_run_num}")
            for tr in st: # a mseed file can have multiple trace, ZNE, 3 traces
                outfile = "{}.{}".format(tr.stats.network,tr.stats.station)
                
                
                #stas is the station information
                stas = read_inventory(f"{data_staxml}/{outfile}.xml")
                
                num_channels = len(stas[0][0])
                print(num_channels)
                for i in range(num_channels):
                    
                    if (tr.stats.channel==stas[0][0][i].code):
                        stas[0][0][i].location_code = tr.stats.location
                        break
                        #print(stas[0][0][i].response)
                
                #print(tr)
#                 elif (tr.stats.channel==stas[0][0][1].code):
#                     stas[0][0][1].location_code = tr.stats.location
#                 else :
#                     stas[0][0][2].location_code = tr.stats.location
                
            # remove the response
                trace = tr.copy()
                #print(trace.start)
                #trace.remove_response(inventory=stas)
                try:
                    trace.remove_response(inventory=stas)
                except:
                    print(f"No matching response information found: {tr}")
                    #print(f"trying to download station data)
                    continue
                #trace.remove_response(inventory=stas) 
                #print("successful run")
                
            # convert to SACTrace from ObsPy Trace
                sactr = SACTrace.from_obspy_trace(trace)
                #sactr is the SAC file/trace
            
            # set event origin time as SAC reference time
                sactr.reftime = evt[1][0]
                sactr.o = 0
                sactr.iztype = 'io'

            # set station location // network, station, channel
                sactr.stla = stas[0][0][0].latitude
                sactr.stlo = stas[0][0][0].longitude
                sactr.stel = stas[0][0][0].elevation
            
            # set event location and magnitude
                sactr.evla = evt[1][1]
                sactr.evlo = evt[1][2]
                sactr.evdp = evt[1][3]
                sactr.mag = evt[1][5]
            
            # set cmpaz and cmpinc
                if (tr.stats.channel==stas[0][0][0].code):
                    sactr.cmpaz = stas[0][0][0].azimuth
                    sactr.cmpinc = stas[0][0][0].dip
                elif (tr.stats.channel==stas[0][0][1].code):
                    sactr.cmpaz = stas[0][0][1].azimuth
                    sactr.cmpinc = stas[0][0][1].dip
                else :
                    sactr.cmpaz = stas[0][0][2].azimuth
                    sactr.cmpinc = stas[0][0][2].dip

            # other SAC headers
                sactr.lcalda = 1  # DIST AZ BAZ and GCARC headers will be calculated from station and event coordinates.
                            # Note that thing may not be same if we use ObsPy Trace.
            
            # write to sac files
                sactr.write(os.path.join(outd, tr.id+'.SAC'))
    print("successful run")

In [15]:
def theo_arrival(data_dir, model_name, phase):
    """
    Write theoretical arrival times of P waves
    """
    total_run_num = len(os.listdir(data_dir))
    run_num = 0
    incomplete_components = 0
    
    model = TauPyModel(model=model_name)

    for ev in os.listdir(data_dir):
        #print(ev)
        run_num+=1
        if run_num%10==0 or run_num ==1:
            print(f"run {run_num} of {total_run_num}")
        inpath = os.path.join(data_dir, ev)

        funiq = []
        for sac in glob.glob(f"{inpath}/*.SAC"):
            st = obspy.read(sac)
            for tr in st:
                funiq.append("{}.{}.{}".format(tr.stats.network,tr.stats.station,tr.stats.location))
        fU = sorted(set(funiq))
        #print(fU)

        for sacfile in fU:
            st = obspy.read(f"{inpath}/{sacfile}*.SAC")
            #print(st)
            sachd = st[0].stats.sac
#             if sachd.get("t1") != None:
#                 continue
            
            
            # set P-wave first arrival
            # distaz is calculating the distance between the station and the event
            
            distaz = client.distaz(sachd["stla"], sachd["stlo"], sachd["evla"], sachd["evlo"])
            gcarc = distaz['distance']
            
            if sachd["evdp"]<0: #when the seismic activity is above sea level
                sachd["evdp"]=0
                
            # using the Tau model to get the time it takes for the phase to arrive using the specific parameters
            arrivals = model.get_travel_times(source_depth_in_km=sachd["evdp"],
                                  distance_in_degree=gcarc, phase_list=[phase])
            print(arrivals)
            print(st)
            os.system.exit()
            
            if len(st)!=3:
                print(st)
                incomplete_components+=1
                continue
            #for i in range(len(st)):
                # t1 is userdefined time marker
                # kt1 is userdefined pick identification
                
#                 st[i].stats.sac["t1"] = arrivals[0].time + sachd["o"]
#                 st[i].stats.sac["kt1"] = arrivals[0].name
            st[0].stats.sac["t1"] = arrivals[0].time + sachd["o"]
            st[0].stats.sac["kt1"] = arrivals[0].name
            st[1].stats.sac["t1"] = arrivals[0].time + sachd["o"]
            st[1].stats.sac["kt1"] = arrivals[0].name
            st[2].stats.sac["t1"] = arrivals[0].time + sachd["o"]
            st[2].stats.sac["kt1"] = arrivals[0].name

            sacf0 = "{}.{}.{}.{}.SAC".format(st[0].stats.network,st[0].stats.station,st[0].stats.location,st[0].stats.channel)
            sacf1 = "{}.{}.{}.{}.SAC".format(st[1].stats.network,st[1].stats.station,st[1].stats.location,st[1].stats.channel)
            sacf2 = "{}.{}.{}.{}.SAC".format(st[2].stats.network,st[2].stats.station,st[2].stats.location,st[2].stats.channel)
            
            st[0].write(f"{inpath}/{sacf0}", format="SAC")
            st[1].write(f"{inpath}/{sacf1}", format="SAC")
            st[2].write(f"{inpath}/{sacf2}", format="SAC")
    print(f"theoretical arrival times of P waves completed, incomplete components: {incomplete_components}")

In [6]:
# dirs and files
wdir = './'
data_mseed = f'{wdir}/Download/miniseed'    
data_staxml = f'{wdir}/Download/stations'
data_sac = f'{wdir}/Download/SAC'                     
evt_lst = f'{wdir}/events.csv'                                     


In [11]:
### change the sensors' record duration

startDate="2022-01-01T00:00:00" 
endDate="2022-07-01T00:00:00"

if not os.path.exists(data_staxml):
    os.makedirs(data_staxml)

for xml in glob.glob(f"{data_staxml}/*.xml"): 
    inv = read_inventory(xml)
    # network
    inv[0].start_date = startDate
    inv[0].end_date = endDate
    # station
    inv[0][0].start_date = startDate
    inv[0][0].end_date = endDate
    num_channels= len(inv[0][0])
    # channel
    for i in range(num_channels):
        inv[0][0][i].start_date = startDate
        inv[0][0][i].end_date = endDate
    # save the modified file
    inv.write(xml, format="stationxml", validate=True)

In [12]:
if not os.path.exists(data_sac):
    os.makedirs(data_sac)
    
# read event metadata
dnames, events = read_evt(evt_lst)

# convert miniseed to sac
mseed2sac(data_mseed, data_sac, data_staxml, dnames, events)

successful run


In [18]:
### Manually check whether the instrument response for a centain station can be found.

from obspy.clients.fdsn import Client
from obspy import UTCDateTime

# specify webservice
client = Client("NCEDC")
starttime = UTCDateTime("1980-01-01")
endtime = UTCDateTime("20021-01-02")
net = "NC"
sta = "CMW"
inv = client.get_stations(network=net, station=sta, channel="HHN,HHE,HHZ", level="response")
print(inv)
name = "{}.{}".format(net,sta)
inv.write(f"{data_staxml}/{name}.xml", format="STATIONXML")

Inventory created at 2022-04-18T18:51:43.000000Z
	Created by: NCEDC WEB SERVICE: fdsnws-station | version: 1.1
		    http://service.ncedc.org/fdsnws/station/1/query?net=NC&sta=CMW&cha=...
	Sending institution: NCEDC (NCEDC)
	Contains:
		Networks (1):
			NC
		Stations (1):
			NC.CMW (Mill Creek)
		Channels (3):
			NC.CMW..HHZ, NC.CMW..HHN, NC.CMW..HHE


In [16]:
# write theoretical arrival times
theo_arrival(data_sac, "iasp91", "ttp")
#ttp is p phase, 

run 1 of 514
7 arrivals
	P phase arrival at 27.637 seconds
	Pn phase arrival at 27.637 seconds
	P phase arrival at 28.359 seconds
	p phase arrival at 28.716 seconds
	P phase arrival at 29.009 seconds
	P phase arrival at 29.331 seconds
	PKiKP phase arrival at 993.898 seconds
3 Trace(s) in Stream:
AZ.LVA2..BHE | 2022-01-02T00:15:14.071703Z - 2022-01-02T00:18:14.081703Z | 100.0 Hz, 18002 samples
AZ.LVA2..BHN | 2022-01-02T00:15:14.071703Z - 2022-01-02T00:18:14.081703Z | 100.0 Hz, 18002 samples
AZ.LVA2..BHZ | 2022-01-02T00:15:14.071703Z - 2022-01-02T00:18:14.081703Z | 100.0 Hz, 18002 samples


AttributeError: 'builtin_function_or_method' object has no attribute 'exit'

In [10]:
def theo_arrival(data_dir, model_name, phase):
    """
    Write theoretical arrival times of P waves
    """
    total_run_num = len(os.listdir(data_dir))
    run_num = 0
    incomplete_components = 0
    
    model = TauPyModel(model=model_name)

    for ev in os.listdir(data_dir):
        #print(ev)
        run_num+=1
        if run_num%10==0 or run_num ==1:
            print(f"run {run_num} of {total_run_num}")
        inpath = os.path.join(data_dir, ev)

        funiq = []
        for sac in glob.glob(f"{inpath}/*.SAC"):
            st = obspy.read(sac)
            for tr in st:
                funiq.append("{}.{}.{}".format(tr.stats.network,tr.stats.station,tr.stats.location))
        fU = sorted(set(funiq))
        #print(fU)

        for sacfile in fU:
            st = obspy.read(f"{inpath}/{sacfile}*.SAC")
            #print(st)
            sachd = st[0].stats.sac
            if sachd.get("t2") != None:
                continue
            print(st)
            
            
            # set P-wave first arrival
            # distaz is calculating the distance between the station and the event
            
            distaz = client.distaz(sachd["stla"], sachd["stlo"], sachd["evla"], sachd["evlo"])
            gcarc = distaz['distance']
            
            if sachd["evdp"]<0: #when the seismic activity is above sea level
                sachd["evdp"]=0
                
            # using the Tau model to get the time it takes for the phase to arrive using the specific parameters
            arrivals = model.get_travel_times(source_depth_in_km=sachd["evdp"],
                                  distance_in_degree=gcarc, phase_list=[phase])
            print(arrivals)
            
            if len(st)!=3:
                print(st)
                incomplete_components+=1
                continue
            #for i in range(len(st)):
                # t1 is userdefined time marker
                # kt1 is userdefined pick identification
                
#                 st[i].stats.sac["t1"] = arrivals[0].time + sachd["o"]
#                 st[i].stats.sac["kt1"] = arrivals[0].name
            st[0].stats.sac["t2"] = arrivals[0].time + sachd["o"]
            st[0].stats.sac["kt2"] = arrivals[0].name
            st[1].stats.sac["t2"] = arrivals[0].time + sachd["o"]
            st[1].stats.sac["kt2"] = arrivals[0].name
            st[2].stats.sac["t2"] = arrivals[0].time + sachd["o"]
            st[2].stats.sac["kt2"] = arrivals[0].name

            sacf0 = "{}.{}.{}.{}.SAC".format(st[0].stats.network,st[0].stats.station,st[0].stats.location,st[0].stats.channel)
            sacf1 = "{}.{}.{}.{}.SAC".format(st[1].stats.network,st[1].stats.station,st[1].stats.location,st[1].stats.channel)
            sacf2 = "{}.{}.{}.{}.SAC".format(st[2].stats.network,st[2].stats.station,st[2].stats.location,st[2].stats.channel)
            
            st[0].write(f"{inpath}/{sacf0}", format="SAC")
            st[1].write(f"{inpath}/{sacf1}", format="SAC")
            st[2].write(f"{inpath}/{sacf2}", format="SAC")
    print(f"theoretical arrival times of P waves completed, incomplete components: {incomplete_components}")
theo_arrival(data_sac, "iasp91", "sP")

run 1 of 514
3 Trace(s) in Stream:
AZ.LVA2..BHE | 2022-01-02T00:15:14.071703Z - 2022-01-02T00:18:14.081703Z | 100.0 Hz, 18002 samples
AZ.LVA2..BHN | 2022-01-02T00:15:14.071703Z - 2022-01-02T00:18:14.081703Z | 100.0 Hz, 18002 samples
AZ.LVA2..BHZ | 2022-01-02T00:15:14.071703Z - 2022-01-02T00:18:14.081703Z | 100.0 Hz, 18002 samples
5 arrivals
	sP phase arrival at 29.212 seconds
	sP phase arrival at 29.694 seconds
	sP phase arrival at 29.704 seconds
	sP phase arrival at 30.415 seconds
	sP phase arrival at 30.481 seconds
3 Trace(s) in Stream:
AZ.RDM..BHE | 2022-01-02T00:15:14.069399Z - 2022-01-02T00:18:14.079399Z | 100.0 Hz, 18002 samples
AZ.RDM..BHN | 2022-01-02T00:15:14.069498Z - 2022-01-02T00:18:14.079498Z | 100.0 Hz, 18002 samples
AZ.RDM..BHZ | 2022-01-02T00:15:14.069399Z - 2022-01-02T00:18:14.079399Z | 100.0 Hz, 18002 samples
5 arrivals
	sP phase arrival at 34.254 seconds
	sP phase arrival at 35.955 seconds
	sP phase arrival at 36.456 seconds
	sP phase arrival at 36.721 seconds
	sP ph

5 arrivals
	sP phase arrival at 33.728 seconds
	sP phase arrival at 35.304 seconds
	sP phase arrival at 35.822 seconds
	sP phase arrival at 35.988 seconds
	sP phase arrival at 36.619 seconds
3 Trace(s) in Stream:
CI.IKP..BHE | 2022-01-02T00:15:14.069498Z - 2022-01-02T00:18:14.079498Z | 100.0 Hz, 18002 samples
CI.IKP..BHN | 2022-01-02T00:15:14.069498Z - 2022-01-02T00:18:14.079498Z | 100.0 Hz, 18002 samples
CI.IKP..BHZ | 2022-01-02T00:15:14.069498Z - 2022-01-02T00:18:14.079498Z | 100.0 Hz, 18002 samples
3 arrivals
	sP phase arrival at 15.502 seconds
	sP phase arrival at 17.080 seconds
	sP phase arrival at 17.082 seconds
3 Trace(s) in Stream:
CI.LRL..BHE | 2022-01-02T00:15:14.069498Z - 2022-01-02T00:18:14.079498Z | 100.0 Hz, 18002 samples
CI.LRL..BHN | 2022-01-02T00:15:14.069498Z - 2022-01-02T00:18:14.079498Z | 100.0 Hz, 18002 samples
CI.LRL..BHZ | 2022-01-02T00:15:14.069498Z - 2022-01-02T00:18:14.079498Z | 100.0 Hz, 18002 samples
5 arrivals
	sP phase arrival at 60.302 seconds
	sP phase a

KeyboardInterrupt: 